1. [Introduction:](#1)
    1. [대회 개요](#2)
    1. [사용 Library](#3)
1. [데이터 확인](#4)
    1. [데이터 셋 확인](#5)
    1. [NULL Data 처리](#6)
    1. [파생변수 생성](#7)
1. [EDA](#8)
    1. [전체 데이터 개요](#9)
    1. [가설 검증](#10)
    1. [ 데이터](#11)
    1. [ 데이터](#12)
1. [데이터 변환](#13)
    1. [Null Data 처리](#14)
    1. [파생변수 생성](#15)
    1. [One-hot Encoding](#16)
1. [모델링 및 평가](#35)
    1. [GLM](#36)
    1. [Light Gradient Boosting](#37)
1. [결론](#38)
    1. [모델 선정](#39)
    1. [추가적인 접근법](#40)

<a id="1"></a> <br>
# 1. INTRODUCTION

<a id="2"></a> <br>
# 1.1 대회 개요

<a id="3"></a> <br>
# 1.2 사용 Library

In [ ]:
# For System
import gc

# For DataFrame
import numpy as np # for linear algebra
import pandas as pd # for data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# For Analysis
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from math import exp
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

# For Visualization
import plotly # visualization
from plotly.graph_objs import Scatter, Figure, Layout # visualization
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot # visualization
import plotly.figure_factory as ff # visualization
import plotly.graph_objs as go # visualization
init_notebook_mode(connected=True) # visualization
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for data visualization
color = sns.color_palette()
%matplotlib inline

<a id="4"></a> <br>
# 2. 데이터

<a id="5"></a> <br>
# 2.1 데이터 셋 확인

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
train_raw = pd.read_csv('../input/train_V2.csv')
test_raw = pd.read_csv('../input/test_V2.csv')

데이터 용량 줄이기
int32 -> int8, 16, 32... etc

In [ ]:

# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    return df


In [ ]:
train_raw.head(10)

In [ ]:
test_raw.head(10)

In [ ]:
# Check dataframe's shape
print('Shape of training set: ', train_raw.shape)
# Types, Data points, memory usage, etc.
print(train_raw.info())


# Check dataframe's shape
print('Shape of test set: ', test_raw.shape)
print(test_raw.info())

<a id="6"></a> <br>
# 2.2 NULL 처리

In [ ]:
train_raw.isnull().sum()

In [ ]:
test_raw.isnull().sum()

train 데이터 셋만 종속변수에 해당하는 "winPlacePerc"에 한 개의 데이터만 Null 값을 가진다.

In [ ]:
print("shape of before train drop null data : "+ str(train_raw.shape[0]) + "," + str(train_raw.shape[1]))
train_raw = train_raw.dropna()
print("shape of after train drop null data : "+ str(train_raw.shape[0]) + "," + str(train_raw.shape[1]))

<a id="7"></a> <br>
# 2.3 파생변수 생성

- **Id** - 플레이어의 Id
- **groupId** - 경기 내의 그룹을 식별하는 ID. 현재 그룹의 선수들이 서로 다른 경기에서 경기한다면, 그들은 매번 다른 groupId를 갖게 될 것이다.
- **matchId** - 경기를 식별하기 위한 ID. train set과 test set에 모두 있는 시합은 없다.
- **matchDuration** - 경기 시간.
- **matchType** - 경기 종류(duo, quad ...).
- **assists** - 팀 동료들과 같이 죽인 적 수이다.
- **boosts** - 부스트 아이템 사용한 수.
- **damageDealt** - 가한 데미지 총량. Note: 자신에게 가한 데미지는 제외.
- **DBNOs** - 빈사상태로 만든 적의 수.
- **headshotKills** - 헤드샷 으로 처리한 적의 수.
- **heals** - 치료 아이템 사용 수.
- **killPlace** - 경기에서 처치한 적의 수 랭킹.
- **killPoints** - 플레이어의 처치 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, killPoints에서 0은 "없음"으로 처리되어야 한다.
- **kills** - 처치한 적의 수.
- **killStreaks** - 단기간에 가장 많이 처치한 적의 최대치.
- **longestKill** - 플레이어가 적을 죽인 가장 긴 거리. 선수를 다운시키고 멀리 운전하는 것이 가장 긴 처치로 이어질 수 있기 때문에 오해의 소지가 있을 수 있다.
- **maxPlace** - 경기에서 가장 순위가 낮은 것에 대한 데이터. 이것은 순위를 건너뛸 수도 있기 때문에 numGroups와 일치하지 않을 수 있다.
- **numGroups** - 경기에 있는 팀의 수.
- **rankPoints** - Elo 방식의 플레이어 랭킹. 다음 버전의 API에서는 삭제될 예정이기 때문에 사용에 주의. '-1' 값은 순위가 "None" 이다.
- **revives** - 플레이어가 팀원 회복 시킨 수.
- **rideDistance** - 차량으로 이동한 거리(단위 : 미터).
- **roadKills** - 차량으로 죽인 플레이어 수.
- **swimDistance** - 수영한 거리(단위 : 미터).
- **teamKills** - 팀킬한 횟수.
- **vehicleDestroys** - 차량을 폭파시킨 횟수.
- **walkDistance** - 걸은 총 거리(단위 : 미터).
- **weaponsAcquired** - 무기 얻은 갯수.
- **winPoints** - 플레이어의 승리 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, winPoints에서 0은 "없음"으로 처리되어야 한다.
- **winPlacePerc** - 예측 목표. 순위의 퍼센트로 표시되며, 1이면 경기에서 1등이고 0이면 경기에서 꼴지 했다는 것이다. 이것은 numGroups로 계산되는게 아니라, maxPlace로 계산되기 때문에 누락되는 것이 있을수도다.

<a id="8"></a> <br>
# 3. EDA

<a id="9"></a> <br>
# 3.1 전체 데이터 개요

In [ ]:
continuous = ['damageDealt','killPlace','killPoints','longestKill','matchDuration','maxPlace','numGroups','rankPoints','rideDistance','walkDistance','winPoints']
discrete = ['assists','boosts','DBNOs','headshotKills','heals','kills','killStreaks','revives','teamKills','vehicleDestroys','weaponsAcquired']
categories = ['matchType']

In [ ]:
### Continuous variable plots
'''
for col in continuous:
    values = train_raw[col].dropna()
    lower = np.percentile(values, 1)
    upper = np.percentile(values, 99)
    fig = plt.figure(figsize=(18,9));
    sns.distplot(values[(values>lower) & (values<upper)], color='Sienna', ax = plt.subplot(121));
    sns.boxplot(y=values, color='Sienna', ax = plt.subplot(122));
    plt.suptitle(col, fontsize=16)
'''

In [ ]:
### Discrete variable plots
'''
NanAsZero = ['']
for col in discrete:
    if col in NanAsZero:
        train_raw[col].fillna(0, inplace=True)
    values = train_raw[col].dropna()  
    fig = plt.figure(figsize=(18,9));
    sns.countplot(x=values, color='Sienna', ax = plt.subplot(121));
    sns.boxplot(y=values, color='Sienna', ax = plt.subplot(122));
    plt.suptitle(col, fontsize=16)
'''

In [ ]:
### Categorical variable plots
'''
for col in categories:
    values = train_raw[col].astype('str').value_counts(dropna=False).to_frame().reset_index()
    if len(values) > 30:
        continue
    values.columns = [col, 'counts']
    fig = plt.figure(figsize=(18,9))
    ax = sns.barplot(x=col, y='counts', color='Sienna', data=values, order=values[col]);
    plt.xlabel(col);
    plt.ylabel('Number of occurrences')
    plt.suptitle(col, fontsize=16)

    ### Adding percents over bars
    height = [p.get_height() for p in ax.patches]    
    total = sum(height)
    for i, p in enumerate(ax.patches):    
        ax.text(p.get_x()+p.get_width()/2,
                height[i]+total*0.01,
                '{:1.0%}'.format(height[i]/total),
                ha="center")    
'''

In [ ]:
### Continuous variable plots
'''
for col in continuous:
    values = train_raw[col].dropna()
    fig = plt.figure(figsize=(18,9));
    sns.jointplot(x=col, y="winPlacePerc", kind="hex", color="#0000FF", data=train_raw)
    plt.suptitle(col, fontsize=16)
'''

damageDealt : 입힌 데미지가 클수록 순위가 높다.

killPlace : 죽인 순위가 높을수록 (0에 가까워질수록) 이긴 순위가 높아진다.

killPoints : 외부랭킹과 경기 순위랑은 유의미한 차이가 보이지 않는다.

longestKill : 단거리로 죽인 사람들은 순위가 낮다.

matchDuration, maxPlace,numGroups, rankPoints, winPoints : 순위와 유의미한 차이가 없다.

In [ ]:
### Discrete variable plots
'''
NanAsZero = ['']
for col in discrete:
    if col in NanAsZero:
        train_raw[col].fillna(0, inplace=True)
    values = train_raw[col].dropna()  
    fig = plt.figure(figsize=(18,9));
    sns.jointplot(x=col, y="winPlacePerc", kind="hex", color="#4CB391", data=train_raw)
    plt.suptitle(col, fontsize=16)
'''

<a id="10"></a> <br>
# 3.2 가설 검증

In [ ]:
train_raw['mvmtDistance'] = train_raw['walkDistance'] + train_raw['rideDistance']+train_raw['swimDistance']

features = list(train_raw.columns)
features.remove("Id")
features.remove("matchId")
features.remove("groupId")
features.remove("matchType")
features.remove("winPlacePerc")

In [ ]:
df_team = train_raw.copy()

df_max= train_raw.groupby(['matchId','groupId'])[features].agg('max')
df_team = pd.merge(train_raw, df_max.reset_index(), suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
df_team = df_team.drop(["assists_max","killPoints_max","headshotKills_max","numGroups_max","revives_max","teamKills_max","roadKills_max","vehicleDestroys_max"], axis=1)


df_rank = df_max.groupby('matchId')[features].rank(pct=True).reset_index()
df_team = pd.merge(df_team, df_rank, suffixes=["", "_maxRank"], how='left', on=['matchId', 'groupId'])
df_team = df_team.drop(["roadKills_maxRank","matchDuration_maxRank","maxPlace_maxRank","numGroups_maxRank"], axis=1)
del df_max
del df_rank
gc.collect()

df_sum = train_raw.groupby(['matchId','groupId'])[features].agg('sum')
df_team = pd.merge(df_team, df_sum.reset_index(), suffixes=["", "_sum"], how='left', on=['matchId', 'groupId'])
df_team = df_team.drop(["assists_sum","killPoints_sum","headshotKills_sum","numGroups_sum","revives_sum","teamKills_sum","roadKills_sum","vehicleDestroys_sum"], axis=1)
del df_sum
gc.collect()

### (1). 열심히 활동한 플레이어가 상위권에 있을 확률이 높다.(존버 X)

In [ ]:
df = train_raw.copy()
df['active'] = df['weaponsAcquired']+df['revives']+df['kills']+df['heals']
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="active", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("active", fontsize=16) 
df[['active','winPlacePerc']].corr()

### (2). 많이 움직인 플레이어가 순위가 높다.

In [ ]:
df = train_raw.copy()
df['mvmtDistance'] = df['walkDistance'] + df['rideDistance']+df['swimDistance']
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="mvmtDistance", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("mvmtDistance", fontsize=16) 
df[['mvmtDistance','winPlacePerc']].corr()

**움직인 거리가 많은수록 승률이 높아진다.**

### (3) 팀전에서 팀플레이를 잘한 사람일수록 승률이 높아진다.

In [ ]:
df = train_raw.copy()
df = df.loc[~df['matchType'].isin(['normal-solo-fpp', 'solo-fpp', 'solo', 'normal-solo'])]
df['teamPlay'] = (df['assists'] +df['revives'])/(df['teamKills']+1)
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="teamPlay", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("teamPlay", fontsize=16) 
df[['teamPlay','winPlacePerc']].corr()

### (4). 많이 움직이고 많이 데미지를 입힌 플레이어의 순위가 높을 것이다.

In [ ]:
df = train_raw.copy()
df['mvmtDistance'] = df['walkDistance'] + df['rideDistance']+df['swimDistance']
df['headshotPercent'] = (df['damageDealt'])*(df['mvmtDistance'])
df = df.loc[df['headshotPercent']>10]
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="headshotPercent", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("headshotPercent", fontsize=16) 
df[['headshotPercent','winPlacePerc']].corr()

### (5). 이동거리 대비 아이템을 많이 먹을수록 순위가 높을 것이다.

In [ ]:
df = train_raw.copy()
df['mvmtDistance'] = df['walkDistance'] + df['rideDistance']+df['swimDistance']
normalized = (df['mvmtDistance']-min(df['mvmtDistance']))/(max(df['mvmtDistance'])-min(df['mvmtDistance']))
df['getitemperMvmt'] = (df['weaponsAcquired'] + df['boosts']+df['heals'])/(normalized+1)

fig = plt.figure(figsize=(18,9));
sns.jointplot(x="getitemperMvmt", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("getitemperMvmt", fontsize=16) 
df[['getitemperMvmt','winPlacePerc']].corr()

### (6). 팀전의 경우 같은 팀의 데미지나 걸은 거리 아이템 얻은 횟수 등의 최고점이 높을수록 순위가 높을 것이다.

In [ ]:
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="mvmtDistance_max", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("mvmtDistance_max", fontsize=16) 
print(df[['mvmtDistance_max','winPlacePerc']].corr())

fig = plt.figure(figsize=(18,9));
sns.jointplot(x="damageDealt_maxRank", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("damageDealt_maxRank", fontsize=16) 
print(df[['damageDealt_maxRank','winPlacePerc']].corr())

### (7). 경기 시간 대비 걸은 거리가 많을수록 순위가 높을 것이다.

In [ ]:
df = train_raw.copy()
df['mvmtDistance'] = df['walkDistance'] + df['rideDistance']+df['swimDistance']
df['distperDuration'] = df['mvmtDistance']/df['matchDuration']

fig = plt.figure(figsize=(18,9));
sns.jointplot(x="mvmtperDuration", y="winPlacePerc", kind="hex", color="#0000FF", data=df)
plt.suptitle("mvmtperDuration", fontsize=16) 
df[['mvmtperDuration','winPlacePerc']].corr()

### (8).이동거리가 적은데 데미지를 많이 입혔으면 순위가 높을 것이다.

In [ ]:
df_team['mvmtDistance'] = df_team['walkDistance'] + df_team['rideDistance']+df_team['swimDistance']
df_team['damageperMvmt'] = (df_team['mvmtDistance']+1)*df_team['damageDealt_maxRank']

fig = plt.figure(figsize=(18,9));
sns.jointplot(x="damageperMvmt", y="winPlacePerc", kind="hex", color="#0000FF", data=df_team)
plt.suptitle("damageperMvmt", fontsize=16) 
df_team[['mvmtDistance','winPlacePerc']].corr()

### (9). 팀의 최고 점수에 따라서 순위가 결정된다.

In [ ]:
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="damageDealt_maxRank", y="winPlacePerc", kind="hex", color="#0000FF", data=df_team)
plt.suptitle("damageDealt_maxRank", fontsize=16) 
df_team[['damageDealt_maxRank','winPlacePerc']].corr()

In [ ]:
fig = plt.figure(figsize=(18,9));
sns.jointplot(x="mvmtDistance_maxRank", y="winPlacePerc", kind="hex", color="#0000FF", data=df_team)
plt.suptitle("mvmtDistance_maxRank", fontsize=16) 
df_team[['mvmtDistance_maxRank','winPlacePerc']].corr()

# Analysis

In [ ]:
df_team = reduce_mem_usage(df_team)
test_raw = reduce_mem_usage(test_raw)

df_pred = df_team.drop(['Id','groupId','matchId'],axis=1)
df_pred = reduce_mem_usage(df_pred)

df_pred = pd.get_dummies(df_pred)
#,'assists','killPoints','kills','killStreaks','longestKill','matchDuration','maxPlace','numGroups','rankPoints','revives','roadKills','swimDistance','teamKills','vehicleDestroys','winPoints'
df_pred_y = df_pred['winPlacePerc']
df_pred_x = df_pred.drop(['winPlacePerc'],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_pred_x, df_pred_y, test_size=0.33, random_state=42)

In [ ]:
params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10
    }


train_set = lgb.Dataset(X_train, y_train, silent=True)
model = lgb.train(params, train_set = train_set, num_boost_round=300)
pred_test_y = model.predict(X_test, num_iteration = model.best_iteration)

rms = sqrt(mean_squared_error(y_test, pred_test_y))
rms

In [ ]:
rms = sqrt(mean_squared_error(y_test, prediction))
print(rms)

df_result = pd.DataFrame(columns=['PRED','REAL'])
df_result['PRED'] = prediction
df_result['REAL'] = y_test.reset_index(drop = True)
df_result.head(100)

In [ ]:
test_raw['mvmtDistance'] = test_raw['walkDistance'] + test_raw['rideDistance']+test_raw['swimDistance']

features = list(test_raw.columns)
features.remove("Id")
features.remove("matchId")
features.remove("groupId")
features.remove("matchType")

df_max= test_raw.groupby(['matchId','groupId'])[features].agg('max')
df_team = pd.merge(test_raw, df_max.reset_index(), suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
df_team = df_team.drop(["assists_max","killPoints_max","headshotKills_max","numGroups_max","revives_max","teamKills_max","roadKills_max","vehicleDestroys_max"], axis=1)


df_rank = df_max.groupby('matchId')[features].rank(pct=True).reset_index()
df_team = pd.merge(df_team, df_rank, suffixes=["", "_maxRank"], how='left', on=['matchId', 'groupId'])
df_team = df_team.drop(["roadKills_maxRank","matchDuration_maxRank","maxPlace_maxRank","numGroups_maxRank"], axis=1)
del df_max
del df_rank
gc.collect()

df_sum = train_raw.groupby(['matchId','groupId'])[features].agg('sum')
df_team = pd.merge(df_team, df_sum.reset_index(), suffixes=["", "_sum"], how='left', on=['matchId', 'groupId'])
df_team = df_team.drop(["assists_sum","killPoints_sum","headshotKills_sum","numGroups_sum","revives_sum","teamKills_sum","roadKills_sum","vehicleDestroys_sum"], axis=1)
del df_sum
gc.collect()

test = df_team.drop(['Id', 'groupId', 'matchId'],axis=1)
test = pd.get_dummies(test)
pred_test_y = model.predict(test, num_iteration = model.best_iteration)

test = pd.DataFrame(columns=['Id', 'winPlacePerc'])
test['Id'] = test_raw['Id']
test['winPlacePerc'] = pred_test_y

test.to_csv('submission.csv',index=False)